In [1]:
!python -m pip install psycopg2-binary apyori

/usr/bin/python: No module named pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg
from apyori import apriori
%matplotlib inline
import getpass

In [3]:
password = getpass.getpass()

 ································································


In [4]:
def pg_connect():
    pg_user = "bgwoiervutgzta"
    pg_pwd = password
    pg_host = "ec2-44-193-228-249.compute-1.amazonaws.com"
    pg_port = 5432
    pg_dbname = "d18l93hamvmtkj"
    return pg.connect(f"dbname='{pg_dbname}' user='{pg_user}' host='{pg_host}' port='{pg_port}' password='{pg_pwd}'")

In [5]:
sql = "SELECT * FROM users join signup_metadata using(user_id)"
df = pd.read_sql(sql, con=pg_connect())


In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df.head(3)

,user_id,user_name,user_email,user_password,user_phone,user_is_admin,signup_md_id,signup_md_device_id,signup_md_paste_count,signup_md_start_date,signup_md_end_date,signup_md_timezone,signup_md_ip,signup_md_os,signup_md_screen_width,signup_md_screen_height,signup_md_device_name,signup_md_browser,signup_md_cpu_cores,signup_md_gpu_name
0,4,Samuel Oliveira,samuel.oliveira@email.com,senhanova,(12) 89887-7626,None,3,98a6045f83298d284b5d20171df903a9,0,2021-09-19 17:16:57.803,2021-09-19 17:17:33.873,America/Sao_Paulo,177.170.125.38,Linux,1366,768,not collected,firefox,8.0,radeon r9 200
1,5,Gabriel Francisco,gabriel.francisco@email.com,senhanova,(12) 97685-5442,None,4,98a6045f83298d284b5d20171df903a9,0,2021-09-19 17:18:26.971,2021-09-19 17:19:07.983,America/Sao_Paulo,177.170.125.38,Linux,1366,768,not collected,firefox,8.0,radeon r9 200
2,6,Teste Aaaa Uau,teste@aaaaaaaa.com.br,1234567890,(12) 98811-2312,None,5,6218ae75e8a1180faf71bbee9b10961d,0,2021-09-19 19:06:20.627,2021-09-19 19:06:20.832,America/Sao_Paulo,,Windows 10,1920,1080,not collected,chrome,8.0,amd radeon vega 8 mobile


In [8]:
df.drop(columns=['signup_md_end_date','signup_md_start_date','signup_md_paste_count','user_phone','user_email','user_name','user_is_admin','signup_md_device_name', 'signup_md_cpu_cores','user_password'], inplace=True)

In [9]:
df.shape

(14, 10)

In [10]:
records = []
for i in range(0, 10): records.append([str(df.values[i,j]) for j in range(0, 10)])

In [11]:
records

[['4',
  '3',
  '98a6045f83298d284b5d20171df903a9',
  'America/Sao_Paulo',
  '177.170.125.38',
  'Linux',
  '1366',
  '768',
  'firefox',
  'radeon r9 200'],
 ['5',
  '4',
  '98a6045f83298d284b5d20171df903a9',
  'America/Sao_Paulo',
  '177.170.125.38',
  'Linux',
  '1366',
  '768',
  'firefox',
  'radeon r9 200'],
 ['6',
  '5',
  '6218ae75e8a1180faf71bbee9b10961d',
  'America/Sao_Paulo',
  '',
  'Windows 10',
  '1920',
  '1080',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['8',
  '6',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['9',
  '7',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['10',
  '8',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'radeon r9 200'],
 ['11',
  '9',
  '98a6045f8329

In [12]:
regras = apriori(records, min_length = 2, min_support = 0.2, min_confidence = 0.2, min_lift = 3)

In [13]:
result = list(regras)

In [14]:
result[0]

RelationRecord(items=frozenset({'1080', '1920'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'1080'}), items_add=frozenset({'1920'}), confidence=1.0, lift=3.3333333333333335), OrderedStatistic(items_base=frozenset({'1920'}), items_add=frozenset({'1080'}), confidence=1.0, lift=3.3333333333333335)])

In [15]:
print(len(result))

538


In [16]:
r = pd.DataFrame(columns=('Items','Antecendete','Consequente','Suporte','Confiança','Lift'))

support=[]
confidence = []
lift =[]
items = []
antecedent = []
consequent = []

for RelationRecord in result:
    for ordered_stat in RelationRecord.ordered_statistics:
        support.append(round(RelationRecord.support,4))
        items.append(RelationRecord.items)
        antecedent.append(ordered_stat.items_base)
        consequent.append(ordered_stat.items_add)
        confidence.append(round(ordered_stat.confidence,4))
        lift.append(round(ordered_stat.lift,4))

r['Items'] = list(map(set,items))
r['Antecendete'] = list(map(set,antecedent))
r['Consequente'] = list(map(set,consequent))
r['Suporte'] = support
r['Confiança'] = confidence
r['Lift'] = lift

r.sort_values(by='Lift', ascending=False, inplace=True)

print(r)

                                                  Items  \
6465  {Linux, 177.170.125.38, 768, 4, firefox, radeo...   
7493  {1080, chrome, 201.46.19.173, America/Sao_Paul...   
7515  {1080, chrome, 201.46.19.173, America/Sao_Paul...   
7514  {1080, chrome, 201.46.19.173, America/Sao_Paul...   
7513  {1080, chrome, 201.46.19.173, America/Sao_Paul...   
...                                                 ...   
6981  {187.34.105.191, 768, 6149ed31bcc27ca1262dd28a...   
6980  {187.34.105.191, 768, 6149ed31bcc27ca1262dd28a...   
6979  {Linux, 187.34.105.191, 768, 6149ed31bcc27ca12...   
6978  {Linux, 187.34.105.191, 768, 6149ed31bcc27ca12...   
0                                          {1080, 1920}   

                                            Antecendete  \
6465                                           {768, 4}   
7493                {1080, 201.46.19.173, chrome, 1920}   
7515          {chrome, 201.46.19.173, Windows 10, 1920}   
7514   {America/Sao_Paulo, 201.46.19.173, chrome, 1920}

In [17]:
pd.set_option('display.max_rows', None)

In [24]:
r.head(10)

,Items,Antecendete,Consequente,Suporte,Confiança,Lift
6465,"{Linux, 177.170.125.38, 768, 4, firefox, radeo...","{768, 4}","{Linux, 177.170.125.38, firefox, radeon r9 200}",0.2,1.0,5.0
7493,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, 201.46.19.173, chrome, 1920}","{America/Sao_Paulo, 771d19f2dd35546be40d4e07bb...",0.2,1.0,5.0
7515,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{chrome, 201.46.19.173, Windows 10, 1920}","{1080, America/Sao_Paulo, 771d19f2dd35546be40d...",0.2,1.0,5.0
7514,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{America/Sao_Paulo, 201.46.19.173, chrome, 1920}","{1080, 771d19f2dd35546be40d4e07bbad1c53, Windo...",0.2,1.0,5.0
7513,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{America/Sao_Paulo, 201.46.19.173, Windows 10,...","{1080, 771d19f2dd35546be40d4e07bbad1c53, chrome}",0.2,1.0,5.0
7508,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, 771d19f2dd35546be40d4e07bbad1c53, chrom...","{America/Sao_Paulo, 201.46.19.173, 1920}",0.2,1.0,5.0
7507,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, America/Sao_Paulo, 771d19f2dd35546be40d...","{201.46.19.173, Windows 10, 1920}",0.2,1.0,5.0
7506,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, America/Sao_Paulo, 771d19f2dd35546be40d...","{201.46.19.173, chrome, 1920}",0.2,1.0,5.0
7505,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, chrome, 201.46.19.173, Windows 10}","{America/Sao_Paulo, 771d19f2dd35546be40d4e07bb...",0.2,1.0,5.0
7504,"{1080, chrome, 201.46.19.173, America/Sao_Paul...","{1080, America/Sao_Paulo, 201.46.19.173, chrome}","{771d19f2dd35546be40d4e07bbad1c53, Windows 10,...",0.2,1.0,5.0


In [19]:
#r.to_csv('apriori.csv')

In [20]:
ids = frozenset(list(df['signup_md_device_id'].unique()))

In [21]:
#Filtering results
alt = pd.DataFrame(columns=('Items','Antecendete','Consequente','Suporte','Confiança','Lift'))

support=[]
confidence = []
lift =[]
items = []
antecedent = []
consequent = []

for RelationRecord in result:
    for ordered_stat in RelationRecord.ordered_statistics:
        if ordered_stat.items_add.issubset(ids): 
            support.append(round(RelationRecord.support,4))
            items.append(RelationRecord.items)
            antecedent.append(ordered_stat.items_base)
            consequent.append(ordered_stat.items_add)
            confidence.append(round(ordered_stat.confidence,4))
            lift.append(round(ordered_stat.lift,4))

alt['Items'] = list(map(set,items))
alt['Antecendete'] = list(map(set,antecedent))
alt['Consequente'] = list(map(set,consequent))
alt['Suporte'] = support
alt['Confiança'] = confidence
alt['Lift'] = lift

alt.sort_values(by='Lift', ascending=False, inplace=True)

In [22]:
alt.head(10)

,Items,Antecendete,Consequente,Suporte,Confiança,Lift
33,"{1080, 771d19f2dd35546be40d4e07bbad1c53, 201.4...","{1080, 201.46.19.173, 1920}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
195,"{chrome, 201.46.19.173, America/Sao_Paulo, 771...","{chrome, 201.46.19.173, America/Sao_Paulo, Win...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
99,"{1080, 201.46.19.173, America/Sao_Paulo, 771d1...","{1080, America/Sao_Paulo, 201.46.19.173, chrome}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
98,"{1080, 201.46.19.173, America/Sao_Paulo, 771d1...","{1080, America/Sao_Paulo, 201.46.19.173, Windo...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
94,"{1080, 201.46.19.173, 771d19f2dd35546be40d4e07...","{1080, 201.46.19.173, chrome, 1920}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
158,"{1080, 201.46.19.173, America/Sao_Paulo, 771d1...","{1080, 201.46.19.173, America/Sao_Paulo, chrom...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
93,"{1080, 201.46.19.173, 771d19f2dd35546be40d4e07...","{1080, 201.46.19.173, Windows 10, 1920}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
92,"{1080, 201.46.19.173, America/Sao_Paulo, 771d1...","{1080, America/Sao_Paulo, 201.46.19.173, 1920}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
37,"{1080, America/Sao_Paulo, 771d19f2dd35546be40d...","{1080, America/Sao_Paulo, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
38,"{1080, 771d19f2dd35546be40d4e07bbad1c53, 201.4...","{1080, 201.46.19.173, Windows 10}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0


In [54]:
alt.to_csv('apriori.csv')

In [59]:
v = set(df.iloc[1].values)
match = lambda rule: rule.ordered_statistics.items_base.intersection(v)
matches = [x for x in map(match, alt['Antecedente'].values) if len(x) > 0]
sorted(matches, key=len, reverse=True)[:5]

AttributeError: 'list' object has no attribute 'items_base'

In [104]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Rule:
    ant: set
    con: set
    
rules = [] 
for row in alt.iloc[:].values:
    rules.append(Data(ant=row[1], con=row[2]))

In [114]:
v = set(df.iloc[3].values)
def match(rule, item):
    # retornar consequencias ou None
    instersection = rule.ant.intersection(set(item))
    if len(instersection) == len(rule.ant):
        return next(iter(rule.con))
matched_rules = [r for r in map(lambda rule: match(rule, v), rules) if r]
matched_rules = set(matched_rules)
matched_rules

{'6149ed31bcc27ca1262dd28a3f3f4ba1'}

{'771d19f2dd35546be40d4e07bbad1c53'}